# Notas a la realización de la práctica

Este documento ofrece algunas indicaciones para la realización de la práctica, que podéis tener en cuenta para evitar errores y para saber cómo recuperar en ciertas situaciones.

## Docker, imágenes y contenedores

Docker permite tener imágenes de pequeñas máquinas virtuales. Estas imágenes son conjuntos de ficheros "inertes" que se pueden utilizar como base para lanzar **contenedores**. Los contenedores ejecutan los ficheros de una imagen para tener una instancia en ejecución (contenedor).

Para saber las imágenes de las que disponemos, podemos hacer `docker images`:

```bash
$ docker images
REPOSITORY               TAG       IMAGE ID       CREATED        SIZE
backupnode-image         latest    ea8e71ca5aee   2 weeks ago    2.18GB
namenode-image           latest    e39efb460f05   2 weeks ago    2.18GB
datanode-image           latest    9f3e878eea85   2 weeks ago    2.18GB
hadoop-base              latest    67efd4a138e9   2 weeks ago    2.06GB
```

Esto muestra las distintas imágenes, su tamaño y cuándo se crearon.

Por otro lado, los contenedores en ejecución se pueden comprobar con `docker ps`. Algunos contenedores estarán en ejecución y otros parados. Si queremos ver todos los contenedores que se ejecutaron en algún momento y que no han sido eliminados, podemos hacer `docker ps -a`:

```bash
$ docker ps -a
CONTAINER ID   IMAGE                    COMMAND                  CREATED       STATUS                     PORTS                    NAMES
e18adb742526   jupyter/scipy-notebook   "tini -g -- start-no…"   4 days ago    Up 5 minutes               0.0.0.0:8888->8888/tcp   practicas-notebook-1
c090609eead9   hadoop-base              "/bin/bash"              2 weeks ago   Exited (0) 4 days ago                               timelineserver
f9436f4c87d9   backupnode-image         "su hdadmin -c 'JAVA…"   2 weeks ago   Exited (143) 4 days ago                             backupnode
f8f93fd2a451   datanode-image           "/inicio.sh"             2 weeks ago   Exited (137) 4 days ago                             datanode4
3d75f2614562   datanode-image           "/inicio.sh"             2 weeks ago   Exited (137) 4 days ago                             datanode1
243ba043e9e7   namenode-image           "/inicio.sh"             2 weeks ago   Exited (137) 4 days ago                             namenode
```


A veces un contenedor no funciona como debería, o hemos cometido un error. Para continuar con la práctica, lo más sencillo es eliminarlo (pararlo primero si no está parado con `docker stop contenedor`) con `docker rm contenedor`. 

Si por el contrario el contenedor está correctamente creado, podemos ponerlo a ejecutar como se especifica en la práctica. En este caso se lanzan a la vez el namenode y los cuatro datanodes:

```bash
docker start namenode datanode{1..4}
```

Al hacer `start`, los servicios del nodo comienzan a ejecutarse de nuevo, y si todos los nodos se ejecutan más o menos al mismo tiempo, la infraestructura de Hadoop hace que se empiecen a ver unos a otros y se pueda utilizar el clúster.

## Servicios dentro de un contenedor

Una vez un contenedor se ha iniciado (con `start` como arriba), se puede "entrar dentro" haciendo uso de la construcción `docker exec` y ejecutando el *shell* de Linux, el bash, teniendo en cuenta pasarle los parámetros `-ti` para que el contenedor mantenga una terminal abierta y podamos interactuar con él:

```bash
$ docker exec -ti namenode bash
```

El contenedor nos mostrará un *prompt* que nos indicará que estamos dentro:

```bash
root@namenode:~# 
```

Nótese que el contenedor se establece en el superusuario por defecto (`root`), y en algún momento deberemos pasar al usuario `hdadmin`:

```bash
# su - hdadmin
```

(el "`-`" se utiliza para que el sistema operativo lea el fichero `~/.bashrc`, que si recordáis, se crea en un momento dado con los valores de las variables que indican dónde está instalado Hadoop).

Esto hará que cambie lo que el *shell* nos muestra y ahora sea:

```bash
hdadmin@namenode:~$
```

Los servicios que se ejecutan dentro de cada contenedor que pertenecen a Hadoop son normalmente servicios java, por lo que con la utilidad `jps` se puede comprobar cuáles tenemos en ejecución. Si no tenemos los servicios que se esperan, puede haber un error en la ejecución o en cómo se construyó el contenedor. Como usuario `hdadmin` (**ESTO ES MUY IMPORTANTE**), hay que ejecutar la utilidad `jps`:

```bash
$ docker exec -ti datanode1 bash
root@datanode1:/# su - hdadmin
hdadmin@datanode1:~$ jps
306 Jps
101 NodeManager
40 DataNode
```

Nótese cómo se cambia al usuario `hdadmin`, y después se ejecuta `jps`. Nótese también cómo en el datanode tienen que estar funcionando el `NodeManager` (gestor del nodo de cara a YARN para asignación de recursos) y el `DataNode` (programa que permite a HDFS almacenar bloques de ficheros).

En el caso del namenode habrá más servicios en ejecución.

Para salir del contenedor hay que escribir `exit` dos veces: una saldrá del usuario `hdadmin` de vuelta hacia `root`, y el último `exit` saldrá del `exec` hacia el contenedor (aunque el contenedor continúa ejecutándose en segundo plano):

```bash
hdadmin@namenode:~$ exit
logout
root@namenode:/# exit
exit
usuario-linux-host:dir$ 
```


## Comprobación por web de los servicios

Recordad que Hadoop ofrece unas direcciones que se pueden acceder a través del navegador. Podéis usarlas para comprobar los nodos disponibles, el estado de HDFS, etc. A veces la actualización de nuevos nodos o nuevos caídos puede tardar hasta 5 o 10 minutos.

- http://localhost:9870 interfaz web del HDFS
- http://localhost:8088 interfaz web de YARN

## Comprobación de los servicios con las utilidades de Hadoop

Normalmente dentro del `namenode`, se pueden usasr las utilidades de Hadoop para comprobar los nodos conectados y el estado de HDFS:

```bash
$ docker exec -ti namenode bash
root@namenode:/# su - hdadmin
hdadmin@namenode:~$ hdfs dfsadmin -report
...
hdadmin@namenode:~$ yarn node -list
...
```


## Refresco de los nodos al añadir o eliminar

En puntos de la práctica, tenéis que añadir y eliminar nodos. Para forzar que Hadoop los reconsidere, hay dos funciones a utilizar, siempre como el usuario `hdadmin`:

```bash
hdadmin@namenode:~$ hdfs dfsadmin -refreshNodes
hdadmin@namenode:~$ yarn rmadmin -refreshNodes
```